[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Geod-Geom/CMfM/blob/main/CMfM.ipynb)

**Environment Set-up**

In [1]:
using_colab = True

# using_colab = False for local jupyter run

if using_colab:
    ! git clone https://github.com/Geod-Geom/CMfM.git
    import sys
    if 'google.colab' in sys.modules:
        import subprocess
        subprocess.call("pip install -U opencv-python".split())
    from google.colab.patches import cv2_imshow
else:
    !pip install -r requirements.txt
    #!pip install opencv-python

fatal: destination path 'CMfM' already exists and is not an empty directory.


**Set-up**

In [ ]:
import os
import glob
import numpy as np
import cv2
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.colorbar as mcolorbar

**Define functions**

In [ ]:
def SIFT_homography(img_reference, img_deformed):

    MIN_MATCH_COUNT = 10
    img1 = cv2.imread(img_reference, 0) # queryImage
    img2 = cv2.imread(img_deformed, 0)  # trainImage

    # Initiate SIFT detector
    sift = cv2.SIFT_create()

    # Find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)
    FLANN_INDEX_KDTREE = 2
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 100)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    # Store all the good matches as per Lowe's ratio test.
    good = []
    coor1 = cv2.KeyPoint_convert(kp1)
    distances = []

    for m,n in matches:
        if m.distance < 0.7*n.distance:
            distances.append(m.distance/n.distance)
            good.append(m)
    print('Good matches', len(good))

    # Draw matches in green color
    draw_params = dict(matchColor = (0,255,0),
                   singlePointColor = None, flags = 2) #cv2.DrawMatchesFlags_DEFAULT)
    img3 = cv2.drawMatches(img1, kp1, img2, kp2, good, None, **draw_params)
    plt.imshow(img3, 'gray'),plt.show()

    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, ransacReprojThreshold = 0.8)

    dst_pts_tras = cv2.perspectiveTransform(src_pts, M)
    distance_euclidean = []
    mask_array = np.zeros((1, len(dst_pts)))

    for il in range(0, len(dst_pts)):
        d = np.sqrt((dst_pts[il][0][0]-dst_pts_tras[il][0][0])**2+(dst_pts[il][0][1]-dst_pts_tras[il][0][1])**2)
        distance_euclidean.append(d)
    distance_array = np.array(distance_euclidean)
    inliers = np.where(mask == 1)[0]
    d_good = distance_array[inliers]

    print('Inlier statistics:')
    print('Max', "{:.2f}".format(np.max(d_good)))
    print('Min', "{:.2f}".format(np.min(d_good)))
    print('Mean', "{:.2f}".format(np.mean(d_good)))
    print('Median', "{:.2f}".format(np.median(d_good)))
    print('Standard deviation', "{:.2f}".format(np.std(d_good)))
    plt.hist(d_good, bins = 100)
    plt.show()

    matchesMask = mask.ravel().tolist()

    h,w = img1.shape
    pts = np.float32([ [0,0], [0,h-1], [w-1,h-1], [w-1,0] ]).reshape(-1, 1, 2)
    dst = cv2.perspectiveTransform(pts,M)
    img2 = cv2.polylines(img2,[np.int32(dst)], True, 255, 3, cv2.LINE_AA)
    warped = cv2.warpPerspective(img1, M, (img1.shape[1], img1.shape[0]))

    # Draw matches in green color
    draw_params = dict(matchColor = (0, 255, 0),
                   singlePointColor = (255, 0, 0),
                   matchesMask = matchesMask, # draw only inliers
                   flags = cv2.DrawMatchesFlags_DEFAULT)
    img3 = cv2.drawMatches(img1, kp1, img2, kp2, good, None, **draw_params)
    plt.imshow(img3, 'gray'),plt.show()

    return M

def template_match(img_reference, img_deformed, method = 'cv2.TM_CCOEFF_NORMED', mlx = 1, mly = 1, show=True):

    # Apply image oversampling
    img_reference = cv2.resize(img_reference, None, fx=mlx, fy=mly, interpolation = cv2.INTER_CUBIC)
    img_deformed  = cv2.resize(img_deformed, None, fx=mlx, fy=mly, interpolation = cv2.INTER_CUBIC)

    res = cv2.matchTemplate(img_deformed, img_reference, eval(method))

    w, h = img_reference.shape[::-1]
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

    # Control if the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum value
    if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
        top_left = min_loc
    else:
        top_left = max_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)

    # Retrieve center coordinates
    px = (top_left[0]+bottom_right[0])/(2.0*mlx)
    py = (top_left[1]+bottom_right[1])/(2.0*mly)

    return px, py, max_val

def CMfM(images_absolute_path, format, img_ind, temp_dim, b, d, ml_x, ml_y, punti):

    # Check if the template dimension is odd
    if int(temp_dim) % 2 == 0:
        temp_dim = temp_dim + 1

    # Find the last folder - the name of the test
    test_name = os.path.basename(os.path.normpath(images_absolute_path))

    msg = test_name + "\n"
    msg = msg + '-----------\n'

    format = '.'+format

    img_names = glob.glob( images_absolute_path +"/*"+format)
    img_names.sort()

    # Read the first image to obtain information about the camera resolution
    img1 = cv2.imread(img_names[0], 0)

    Dim_x = np.shape(img1)[1] # width
    Dim_y = np.shape(img1)[0] # height

    msg = ''
    msg = msg + '-----------'
    msg = msg +  "\nAnalysed images"
    msg = msg +  "\n1: "+ os.path.basename(str(img_names[0]))
    msg = msg +  "\n2: "+ os.path.basename(str(img_names[img_ind]))
    print(msg)

    img1 = cv2.imread(img_names[0], 0)
    img2 = cv2.imread(img_names[img_ind], 0)

    img1r = img1.copy()
    img2r = img2.copy()

    # Compute homography
    H_homography = SIFT_homography(img_names[0], img_names[img_ind])
    H_homography_inv = np.linalg.inv(H_homography)

    # Read points of interest along the crack
    I_sinistra_crack = punti[:,0]
    J_sinistra_crack = punti[:,1]
    I_destra_crack = punti[:,2]
    J_destra_crack = punti[:,3]
    I_PUNTI = np.hstack((I_sinistra_crack, I_destra_crack))
    J_PUNTI = np.hstack((J_sinistra_crack, J_destra_crack))
    PUNTI = np.vstack((I_PUNTI, J_PUNTI)).T

    # Apply homography to the points of interest
    pt = PUNTI
    PUNTI_MASTER_I = []
    PUNTI_MASTER_J = []
    PUNTI_SLAVE_I = []
    PUNTI_SLAVE_J = []
    PTS_trans_LMEDS_list = []
    dx_homo = []
    dy_homo = []

    # Template matching over the selected points
    for pp in PUNTI:

        TP_temp_x_P = pp[0] # x start
        TP_temp_y_P = pp[1] # y start

        start_x_template_slice1 = int(TP_temp_x_P) - int((temp_dim-1)/2.0)
        stop_x_template_slice1  = int(TP_temp_x_P) + int((temp_dim-1)/2.0)
        start_y_template_slice1 = int(TP_temp_y_P) - int((temp_dim-1)/2.0)
        stop_y_template_slice1  = int(TP_temp_y_P) + int((temp_dim-1)/2.0)

        shape_template1 = np.shape( img1r [start_y_template_slice1 : stop_y_template_slice1+1 , start_x_template_slice1 : stop_x_template_slice1+1])

        # Check the template dimensions
        assert np.allclose(shape_template1[0], temp_dim)
        assert np.allclose(shape_template1[1], temp_dim)
        assert np.allclose(stop_y_template_slice1+1 - start_y_template_slice1, temp_dim)
        assert np.allclose(stop_x_template_slice1+1 - start_x_template_slice1, temp_dim)
        assert np.allclose(TP_temp_x_P, (start_x_template_slice1+ stop_x_template_slice1+1)//2.0)
        assert np.allclose(TP_temp_y_P, (start_y_template_slice1+ stop_y_template_slice1+1)//2.0)

        PTS1 = np.vstack((float(TP_temp_x_P), float(TP_temp_y_P))).T
        PTS = np.array([PTS1])

        PTS_trans_LMEDS = cv2.perspectiveTransform(PTS, H_homography)

        PTS_trans_LMEDS_list.append(PTS_trans_LMEDS)

        start_x_search_slice1 = int(PTS_trans_LMEDS[0][0][0] - (temp_dim-1)/2.0 - d)
        stop_x_search_slice1  = int(PTS_trans_LMEDS[0][0][0] + (temp_dim-1)/2.0 + d)
        start_y_search_slice1 = int(PTS_trans_LMEDS[0][0][1] - (temp_dim-1)/2.0 - b)
        stop_y_search_slice1  = int(PTS_trans_LMEDS[0][0][1] + (temp_dim-1)/2.0 + b)
        shape_search1 = np.shape( img2r [  start_y_search_slice1 : stop_y_search_slice1+1 , start_x_search_slice1 : stop_x_search_slice1+1 ])

        # Check the search area dimensions
        assert np.allclose((shape_search1[0] - temp_dim) /2.0, b)
        assert np.allclose((shape_search1[1] - temp_dim) /2.0, d)
        assert np.allclose(int(PTS_trans_LMEDS[0][0][0]), (start_x_search_slice1 + stop_x_search_slice1+1)//2.0)
        assert np.allclose(int(PTS_trans_LMEDS[0][0][1]), (start_y_search_slice1 + stop_y_search_slice1+1)//2.0)

        temp1 =        img1r [  start_y_template_slice1 : stop_y_template_slice1+1 , start_x_template_slice1 : stop_x_template_slice1+1 ]
        search_area1 = img2r [  start_y_search_slice1 : stop_y_search_slice1+1 , start_x_search_slice1 : stop_x_search_slice1+1 ]

        indx1,indy1, maxcc1 = template_match(temp1.astype('uint8'), search_area1.astype('uint8'), mlx = ml_x, mly = ml_y, show = False)

        TP_search_x_P = start_x_search_slice1+indx1 - 0.5   # end point x
        TP_search_y_P = start_y_search_slice1+indy1 - 0.5   # end point y
        print("Matching results: ")
        print('i, j, coeff cross: ', "{:.2f}".format(TP_search_x_P), "{:.2f}".format(TP_search_y_P), "{:.2f}".format(maxcc1))

        # Append point positions of master and slave images
        PUNTI_MASTER_I.append(TP_temp_x_P)     # start point x [pixel]
        PUNTI_MASTER_J.append(TP_temp_y_P)     # start point y [pixel]
        PUNTI_SLAVE_I.append(TP_search_x_P)    # end point x [pixel]
        PUNTI_SLAVE_J.append(TP_search_y_P)    # end point y [pixel]

        dx_homo.append(TP_search_x_P- PTS_trans_LMEDS[0][0][0])
        dy_homo.append(TP_search_y_P- PTS_trans_LMEDS[0][0][1])

    return H_homography, H_homography_inv, dx_homo, dy_homo, PUNTI_MASTER_I, PUNTI_MASTER_J, PUNTI_SLAVE_I, PUNTI_SLAVE_J, PTS_trans_LMEDS_list

**Data import**

In [ ]:
if using_colab:
    path = "/content/CMfM/images/"
    os.mkdir('/content/CMfM/results/')
    path_results = '/content/CMfM/results/'
    points = np.genfromtxt("/content/CMfM/crack_point.txt", skip_header=1)
else:
    path = "images/"
    os.mkdir('results/')
    path_results = 'results/'
    points = np.genfromtxt("crack_point.txt", skip_header=1)

path_moving = path + 'moving/'
path_fix = path + 'fix/'
path_clahe = path + 'clahe/'

data_path = os.path.join(path_moving,'*.jpg')
files = sorted(glob.glob(data_path))

data_path_fix = os.path.join(path_fix,'*.jpg')
files_fix = sorted(glob.glob(data_path_fix))

**Show fixed reference and deformed images**

In [ ]:
fixed_ref = cv2.imread(files_fix[0])
fixed_deformed = cv2.imread(files_fix[1])

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16,28))

# plots
ax[0].imshow(fixed_ref)
ax[0].set_title("Fixed reference image")
ax[1].imshow(fixed_deformed)
ax[1].set_title("Fixed deformed image")

**Show moving reference and deformed images**

In [ ]:
fixed_ref = cv2.imread(files[0])
fixed_deformed = cv2.imread(files[1])

fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(16,28))

# plots
ax[0].imshow(fixed_ref)
ax[0].set_title("Moving reference image")
ax[1].imshow(fixed_deformed)
ax[1].set_title("Moving deformed image 1")

fixed_ref = cv2.imread(files[0])
fixed_deformed = cv2.imread(files[2])

fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(16,28))

# plots
ax[0].imshow(fixed_ref)
ax[0].set_title("Moving reference image")
ax[1].imshow(fixed_deformed)
ax[1].set_title("Moving deformed image 2")

fixed_ref = cv2.imread(files[0])
fixed_deformed = cv2.imread(files[3])

fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(16,28))

# plots
ax[0].imshow(fixed_ref)
ax[0].set_title("Moving reference image")
ax[1].imshow(fixed_deformed)
ax[1].set_title("Moving deformed image 3")

**Set-up matching parameters**

In [ ]:
ml_x = 20.0
ml_y = 20.0

# Moving camera
templ_mov = 21
edge_mov_x = 30
edge_mov_y = 30

# Fixed camera
templ_fix = 81
edge_fix_x = 60
edge_fix_y = 60

**Run main for standard DIC and CMfM computation**

In [ ]:
# Compute homography between reference fixed and moving cameras
H_homography_fix = SIFT_homography(path_clahe + "clahe_1341_cropped.jpg", path_clahe + "clahe_02051_cropped.jpg")

# Carry out CMfM and DIC on moving and fixed cameras
for tt in range(1, len(files)):
    img_reference = files[0]
    img_deformed = files[tt]

    H_homography, H_homography_inv, dx_homo, dy_homo, PUNTI_MASTER_I, PUNTI_MASTER_J, PUNTI_SLAVE_I, PUNTI_SLAVE_J, PTS_trans_LMEDS_list = CMfM(path_moving, "jpg", tt, templ_mov, edge_mov_x, edge_mov_y, ml_x, ml_y, points)

    # Stack of matched points
    dx_dy = np.vstack((PUNTI_SLAVE_I, PUNTI_SLAVE_J)).T

    # Apply inverse of homography to project the slave points on the master image
    pt_dx_dy = dx_dy
    pts_dx_dy = np.array([pt_dx_dy])
    dx_dy_trasf_imm1 = cv2.perspectiveTransform(pts_dx_dy, H_homography_inv)
    n_points = len(points[:,0])

    # Residual differences
    dx_punti = dx_dy_trasf_imm1[0][:,0]
    dy_punti = dx_dy_trasf_imm1[0][:,1]
    Punti_crack_sinistradx = dx_punti[0:n_points]
    Punti_crack_destradx = dx_punti[n_points:]
    Punti_crack_sinistrady = dy_punti[0:n_points]
    Punti_crack_destrady = dy_punti[n_points:]

    ## Draw the selected points of interest on the master image
    PUNTI = np.vstack((PUNTI_MASTER_I, PUNTI_MASTER_J)).T

    img = cv2.imread(path_moving + img_reference)
    for pointx, pointy in zip(PUNTI[:,0], PUNTI[:,1]):
        cv2.circle(img, (int(pointx), int(pointy)), 30,(255,0,0), 9)
    #cv2_imshow(img)

    # Draw point positions coming from homography (blue) and results of template matching (green)
    PUNTI_slave = np.vstack((PUNTI_SLAVE_I, PUNTI_SLAVE_J)).T
    A = np.array(PTS_trans_LMEDS_list).flatten()
    AA = np.reshape(A, (n_points*2, 2))

    img = cv2.imread(img_deformed)

    for pointx, pointy in zip(AA[:,0], AA[:,1]):
        cv2.circle(img, (int(pointx), int(pointy)), 30,(0,0,255), 9)

    cv2.imwrite(path_results +str(tt)+"point.jpg", img)

    I_sinistra_crack = points[:,0].astype(float)
    J_sinistra_crack = points[:,1].astype(float)
    I_destra_crack = points[:,2].astype(float)
    J_destra_crack = points[:,3].astype(float)
    sinistra = np.vstack((I_sinistra_crack, J_sinistra_crack)).T
    destra = np.vstack((I_destra_crack, J_destra_crack)).T
    pt = sinistra
    pts_sn = np.array([pt])
    pt1_LMEDS = cv2.perspectiveTransform(pts_sn, H_homography_fix)
    pt_ds = destra
    pts_ds = np.array([pt_ds])
    pt1_LMEDS_ds = cv2.perspectiveTransform(pts_ds, H_homography_fix)

    M_inv_fix = np.linalg.inv(H_homography_fix)

    P1_txt = pt1_LMEDS[0]
    P2_txt = pt1_LMEDS_ds[0]
    P1x = P1_txt[:,0]
    P1y = P1_txt[:,1]
    P2x = P2_txt[:,0]
    P2y = P2_txt[:,1]

    Matched_sinistra_MOVING = []
    Matched_destra_MOVING = []
    Matched_sinistra_FIXED = []
    Matched_destra_FIXED = []

    for iii in range(0,len(P1x)):

        #Point n.1
        x_point= round(P1x[iii])
        y_point= round(P1y[iii])

        templ_fix = 81

        px = int(x_point-(templ_fix-1)/2)
        py = int(y_point-(templ_fix-1)/2)

        edge_fix_x = 60
        edge_fix_y = 60

        img_reference_fix = cv2.imread(files_fix[0], 0)
        img_deformed_fix = cv2.imread(files_fix[1], 0)

        UL_I = px-edge_fix_x
        UL_J = py-edge_fix_y

        DIM_I = 2*edge_fix_x+templ_fix
        DIM_J = 2*edge_fix_y+templ_fix

        temp_1 = img_reference_fix[py : py+templ_fix, px : px+templ_fix]
        search_area_1 = img_deformed_fix[UL_J : UL_J+DIM_J, UL_I : UL_I+DIM_I]

        i, j, maxcc = template_match(temp_1, search_area_1, mlx = ml_x, mly = ml_y)
        print("Matching results:")
        print('i, j, coeff cross: ', i,j, "{:.2f}".format(maxcc))

        #Point n.2
        x_point_2= round(P2x[iii])
        y_point_2= round(P2y[iii])

        px_2 = int(x_point_2-(templ_fix-1)/2)
        py_2 = int(y_point_2-(templ_fix-1)/2)

        UL_I_2 = px_2-edge_fix_x
        UL_J_2 = py_2-edge_fix_y
        DIM_I_2 = 2*edge_fix_x+templ_fix
        DIM_J_2 = 2*edge_fix_y+templ_fix

        temp_2= img_reference_fix[py_2 : py_2+templ_fix, px_2 : px_2+templ_fix,]
        search_area_2 = img_deformed_fix[UL_J_2 : UL_J_2+DIM_J, UL_I_2 : UL_I_2+DIM_I]

        i_2, j_2, maxcc_2 = template_match(temp_2, search_area_2, mlx = ml_x, mly = ml_y)
        print("Matching results:")
        print('i, j, coeff cross: ', i_2, j_2, "{:.2f}".format(maxcc_2))
        search_I = P1x[iii]-(templ_fix-1)/2-edge_fix_x
        search_J = P1y[iii]-(templ_fix-1)/2-edge_fix_y
        search_I2 = P2x[iii]-(templ_fix-1)/2-edge_fix_x
        search_J2 = P2y[iii]-(templ_fix-1)/2-edge_fix_y

        Isinistra=((search_I+i)-0.5)
        Idestra=((search_I2+i_2)-0.5)
        Jsinistra=((search_J+j)-0.5)
        Jdestra=((search_J2+j_2)-0.5)
        P_sinistra = np.vstack((Isinistra, Jsinistra)).T

        pt_dx_dy_sin = P_sinistra
        pts_dx_dy_sinistra = np.array([pt_dx_dy_sin])
        dx_dy_trasf_sinistra = cv2.perspectiveTransform(pts_dx_dy_sinistra, M_inv_fix)

        P_destra = np.vstack((Idestra, Jdestra)).T

        Matched_sinistra_FIXED.append([P_sinistra])
        Matched_destra_FIXED.append([P_destra])

        pt_dx_dy_des = P_destra
        pts_dx_dy_destra = np.array([pt_dx_dy_des])
        dx_dy_trasf_destra = cv2.perspectiveTransform(pts_dx_dy_destra, M_inv_fix)

        Matched_sinistra_MOVING.append([dx_dy_trasf_sinistra])
        Matched_destra_MOVING.append([dx_dy_trasf_destra])

    POINTS_FIX = np.hstack((pt1_LMEDS, pt1_LMEDS_ds))

    img = cv2.imread(files_fix[0])
    for pointx, pointy in zip(POINTS_FIX[0][:,0], POINTS_FIX[0][:,1]):
        cv2.circle(img, (int(pointx), int(pointy)), 30,(0,0, 255), 9)

    cv2.imwrite(path_results +"point_IMG_02051.jpg", img)

    final_point_moving_master_from_fixed_sn = np.array(list(plt.cbook.flatten(Matched_sinistra_MOVING))).reshape(len(points[:,0]), 2)

    fix_sn_dx = final_point_moving_master_from_fixed_sn[:,0]
    fix_sn_dy = final_point_moving_master_from_fixed_sn[:,1]
    inizial_point_moving_master_sn = np.array(list(plt.cbook.flatten(Matched_sinistra_MOVING))).reshape(len(points[:,0]), 2)
    final_point_moving_master_from_fixed_ds = np.array(list(plt.cbook.flatten(Matched_destra_MOVING))).reshape(len(points[:,0]), 2)
    fix_ds_dx = final_point_moving_master_from_fixed_ds[:,0]
    fix_ds_dy = final_point_moving_master_from_fixed_ds[:,1]
    inizial_point_moving_master_ds = np.array(list(plt.cbook.flatten(Matched_destra_MOVING))).reshape(len(points[:,0]), 2)
    fixed = final_point_moving_master_from_fixed_ds-final_point_moving_master_from_fixed_sn
    crack_dx_moving = Punti_crack_destradx - Punti_crack_sinistradx
    crack_dx_fixed = fix_ds_dx - fix_sn_dx

    crack_dy_moving = Punti_crack_destrady - Punti_crack_sinistrady
    moving = np.vstack((crack_dx_moving, crack_dy_moving)).T

    results = -crack_dx_moving + crack_dx_fixed
    fixed_mod = np.sqrt(fixed[:,0]**2+fixed[:,1]**2)
    moving_mod = np.sqrt(moving[:,0]**2+moving[:,1]**2)
    results_mod = -moving_mod +fixed_mod

    crack_finale_mov = crack_dx_moving-(points[0][2]-points[0][0])
    crack_finale_fix = crack_dx_fixed-(points[0][2]-points[0][0])
    plt.plot(crack_finale_mov, 'bo', label = "DfM - Moving camera")
    plt.plot(crack_finale_fix, 'ro', label = "DIC - Fixed camera")
    plt.xlabel("Points")
    plt.ylabel("Crack width [px]")
    plt.legend(loc='upper left')
    plt.ylim(0,30)
    plt.savefig(path_results + str(tt)+'crack_monitoring.pdf')
    plt.show()

    crack_finale = crack_dx_moving-(points[0][2]-points[0][0])
    np.savetxt(path_results + 'test'+str(tt)+'.txt', crack_finale, delimiter=',')
    plt.plot(results, 'go', label = "Fixed camera - Moving camera")
    plt.ylim(-4,4)
    plt.xlabel("Points")
    plt.ylabel("Difference [px]")
    plt.legend(loc='upper left')
    plt.savefig(path_results + str(tt)+'errors.pdf')
    plt.show()